### Optimización

$$argmin_{q} | ( \mathbf{q} - \mathbf{q_{0}} ) \oslash  \mathbf{q_{0}} |^{2} \;,\; | ( \mathbf{q} - \mathbf{q_{0}} ) \oslash  \mathbf{q_{0}} |^{\infty}
\\subject\to: \quad {Rq = b \;, \\ Mq \leq d}$$


In [3]:
import cvxpy as cp
import numpy as np
import time

In [4]:
def evaluate_sol(x):
    """Recieves solution vector and computes difference w.r.t PRESUPUESTO and HISTORICO"""

    global diffs_prod, diffs_hist
    assert np.all(x > 0)
    
    diffs_prod = [sum([x[n] for n in Q[i,:,:].flatten()]) - PRES[i] for i in range(p)]
    
    diffs_hist = [x[n] - HIST[n] for n in range(t)]
    
    return np.around([np.sum(np.abs(diffs_prod)), np.max(diffs_prod), np.min(diffs_prod), np.mean(diffs_prod), 
                      np.std(diffs_prod), np.mean(diffs_hist), np.std(diffs_hist)], 4)

In [5]:
def solve(p, d, c, PRES, HIST, solver = 'SCS', o = None, _ = 1):
    # Vamos a optimizar para producto, depósito y canal
    # Hay aprox 139 productos, 17 depósitos y 3 canales
    
    # Vector con p*d*c variables
    global x, Q

    cons = []

    # todo elemento es mayor a 0
    for e in x:
        cons.append(e >= 0)

    # para el producto i:
    # sum(Q_idc) <= PRES[i]
    if _:
        for i in range(p):
            cons.append(sum([x[n] for n in Q[i,:,:].flatten()]) <= PRES[i])
    else:
        for i in range(p):
            cons.append(sum([x[n] for n in Q[i,:,:].flatten()]) == PRES[i])
    
    # funcion objetivo, minimizar la diferencia con el mes anterior (HIST) 
    # y maximizar la asignación de productos 

    if o:
        obj = cp.Minimize(o)
    else: 
        obj = cp.Minimize(cp.sum_squares(x - HIST) - sum(x))

    prob = cp.Problem(obj, cons)
    
    t0 = time.time()
    sol = prob.solve('SCS')
    t1 = time.time()
    
    S = np.fromiter(x.value, dtype = 'float32')
    
    _ = evaluate_sol(S)
    
    return list(_) + [np.around(t1-t0), prob.status]

In [6]:
np.random.seed(101)
p, d, c = 139, 17, 3
t = p*d*c
PRES = np.random.randint(10000, 20000, size = p)
HIST = np.floor(np.random.normal(PRES.sum()//t, size = t))

# Tensor con una var para cada producto+deposito+canal
Q = np.arange(t).reshape((p,d,c))
x = cp.Variable(t)


solvers =  ['CVXOPT', 'GLPK', 'GLPK_MI', 'SCS', 'LS']

objs = [cp.sum_squares(x - HIST), cp.sum_squares(x - HIST) - sum(x), cp.sum_squares(x - HIST) - 2*sum(x), 
        cp.sum_squares(x - HIST) - 3*sum(x), cp.sum_squares(x - HIST) - cp.sum_squares(x)]

In [ ]:
%%time

d = {}
for m in solvers:
    for j in range(len(objs)):
        for z in [0, 1]:
            try:
                d[(m, str(j), ['==', '<='][z])] = solve(p, d, c, PRES, HIST, m, objs[j], z)
            except:
                d[(m, str(j), ['==', '<='][z])] = None

In [ ]:
import pickle

path = input('out')
with open(path + '.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pandas as pd

for k in d:
    d[str(k)] = d[k]
    del d[k]
df = pd.DataFrame.from_dict(d)

In [19]:
s = solve(p, d, c, PRES, HIST)

In [69]:
df

,"(SCS, 3, ==)","(SCS, 3, <=)","(SCS, 4, ==)","(SCS, 4, <=)","(LS, 0, ==)","(LS, 0, <=)","(LS, 1, ==)","(LS, 1, <=)","(LS, 2, ==)","(LS, 2, <=)",...,"('GLPK_MI', '3', '==')","('GLPK_MI', '3', '<=')","('GLPK_MI', '4', '==')","('GLPK_MI', '4', '<=')","('SCS', '0', '==')","('SCS', '0', '<=')","('SCS', '1', '==')","('SCS', '1', '<=')","('SCS', '2', '==')","('SCS', '2', '<=')"
ΔPRESsum,None,None,None,None,2769.46,148865,1002.66,67417.4,687.274,8971.06,...,None,None,None,None,2769.46,148865,1002.66,67417.4,687.274,8971.06
ΔPRESmax,None,None,None,None,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548,...,None,None,None,None,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548
ΔPRESmin,None,None,None,None,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191,...,None,None,None,None,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191
ΔPRESmean,None,None,None,None,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54,...,None,None,None,None,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54
ΔPRESstd,None,None,None,None,22.3782,1502.37,6.2227,939.804,3.0349,186.104,...,None,None,None,None,22.3782,1502.37,6.2227,939.804,3.0349,186.104
ΔHISTmean,None,None,None,None,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282,...,None,None,None,None,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282
ΔHISTstd,None,None,None,None,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968,...,None,None,None,None,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968
t,None,None,None,None,42,42,47,47,46,47,...,None,None,None,None,42,41,45,45,46,46
status,None,None,None,None,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,...,None,None,None,None,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate


In [57]:
df.index = 'ΔPRESsum ΔPRESmax  ΔPRESmin  ΔPRESmean  ΔPRESstd  ΔHISTmean  ΔHISTstd  t status'.split()

In [ ]:
df.iloc[3, 4]

In [60]:
df.dropna(axis = 1)[:, df.iloc[3]]

,"(LS, 0, ==)","(LS, 0, <=)","(LS, 1, ==)","(LS, 1, <=)","(LS, 2, ==)","(LS, 2, <=)","('CVXOPT', '0', '==')","('CVXOPT', '0', '<=')","('CVXOPT', '1', '==')","('CVXOPT', '1', '<=')",...,"('GLPK_MI', '1', '==')","('GLPK_MI', '1', '<=')","('GLPK_MI', '2', '==')","('GLPK_MI', '2', '<=')","('SCS', '0', '==')","('SCS', '0', '<=')","('SCS', '1', '==')","('SCS', '1', '<=')","('SCS', '2', '==')","('SCS', '2', '<=')"
ΔPRESsum,2769.46,148865,1002.66,67417.4,687.274,8971.06,2769.46,148865,1002.66,67417.4,...,1002.66,67417.4,687.274,8971.06,2769.46,148865,1002.66,67417.4,687.274,8971.06
ΔPRESmax,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548,32.6655,-3.3467,5.0683,-2.4308,...,5.0683,-2.4308,0.4196,-1.548,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548
ΔPRESmin,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191,-45.921,-4595.31,-16.7841,-3044.69,...,-16.7841,-3044.69,-10.2376,-857.191,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191
ΔPRESmean,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54,-6.6363,-1070.97,-5.8607,-485.017,...,-5.8607,-485.017,-4.9102,-64.54,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54
ΔPRESstd,22.3782,1502.37,6.2227,939.804,3.0349,186.104,22.3782,1502.37,6.2227,939.804,...,6.2227,939.804,3.0349,186.104,22.3782,1502.37,6.2227,939.804,3.0349,186.104
ΔHISTmean,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282,1.0071,-19.8622,1.0223,-8.3729,...,1.0223,-8.3729,1.041,-0.1282,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282
ΔHISTstd,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968,55.5141,32.6094,55.1974,42.7819,...,55.1974,42.7819,55.1348,53.1968,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968
t,42,42,47,47,46,47,46,49,61,61,...,46,43,45,45,42,41,45,45,46,46
status,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,...,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate


In [1]:
def solve_cvx(A, x, b, idx_ineqs, solver = 'SUPERSCS', max_iters=100, eps = 1e-09, scale=1):
    n = x.shape[0]
    q1 = cp.Variable(n, nonneg=True)
    #warm start 
    q1.value = x

    constrains=[]
    for i in range(A.shape[0]):
        if i in idx_ineqs:
            constrain = cp.sum(q1[A[i].toarray().ravel()]) <= int(b[i])
        else:
            constrain = cp.sum(q1[A[i].toarray().ravel()]) == int(b[i])        
        constrains.append(constrain)
    
    print('Constrains loaded.')
    
    obj = cp.Minimize(cp.norm(q1-x, p=1))
    #obj = cp.Minimize(cp.norm(cp.kl_div(x, q1)))# - cp.sum(q1)))
    prob = cp.Problem(obj, constrains)
    print("Problem made.")

    # liberar memoria
    del A, x, b

    if solver == 'OSQP':
        sol = prob.solve(solver, warm_start = True, verbose = True, max_iter=max_iters, linsys_solver='mkl pardiso')
    elif 'SCS' in solver:
        sol = prob.solve(solver, warm_start = True, verbose = True, max_iters=max_iters, eps=eps, scale=scale)
    print("Problem solved.")

    return q1.value, sol

In [2]:
def solve_rel(A, q, b, idx_ineqs):
    """En esta solución, se encuentra el vector más cercano a q que cumple con las restricciones,
    usando como producto interno una matriz con el inverso de q en la diagonal. 
    Resulta en diferencias relativas al valor correspondiente inicial de q."""
    
    #indices de igualdades
    eqs = [n for n in range(b.shape[0]) if n not in idx_ineqs.astype(int)]
    #resultados actuales
    b_ = A @ q
    # Se les asigna a las inecuaciones incumplidas el valor maximo permitido
    idx_ineqs_ok = np.intersect1d(np.where((b - b_) > 0), idx_ineqs).astype(int)
    b[idx_ineqs_ok] = b_[idx_ineqs_ok]
    
    #matrix diagonal (sparse)
    W = diags(q)
    #least squares
    sol, *_ = lsqr(A, b)
    #de var a subesp
    o = q - sol
    #base ortogonal
    ort = orth((A @ W).T.todense())
    base = W @ ort
    x = o - base @ (base.T @ (W.power(-2) @ o))
    q_ = x + sol
    return q_

In [47]:
%%time
def solve_lstsq():
    b[idx_ineqs[(A @ x)[idx_ineqs] <= b[idx_ineqs]]] = (A @ x)[idx_ineqs[(A @ x)[idx_ineqs] <= b[idx_ineqs]]]
    x = np.linalg.lstsq(A.todense(), b)
    write_sol(x)
    validar()

Wall time: 0 ns


In [187]:
def perturbar(ineq=25000000):
    b = np.loadtxt(TMP_PATH+ 'b.gz').astype('float32')
    b[0] = ineq
#    d = np.random.uniform(0.8, 1, size=b.shape[0])
#    b *= d
    np.savetxt(TMP_PATH + r'\b.gz', b)

    np.set_printoptions(suppress=True)
    print('Diferencias con objetivos,', (A @ q - b)[:10], 
          'Resultados totales,', (A @ q)[:10], sep='\n')

In [85]:
def solve_nras(A, q, b, idx_ineqs, precision=10):
    """Devuelve (b - (A @ q)) """
    c = 0
    d_ = np.inf
    while True:
        p = (b / (A @ q))
        idx_ineqs_ok = np.where(p[idx_ineqs] > 1 + 1e-10)
        p[idx_ineqs_ok] = 1
        A_= A.multiply(p[:, np.newaxis])
        P = np.squeeze(np.array(np.true_divide(A_.sum(0), (A_!=0).sum(0))))
        q = q * P
        d = np.linalg.norm((b - (A @ q))[[idx for idx in range(A.shape[0]) if idx not in idx_ineqs_ok]])
        print(d, end='\t')
        if d >= d_:
            c += 1
        d_ = d
        if (c == 10) | (d_ < precision):
            break
    return q

In [66]:
def solve_ultra(A, q0, b, idx_ineqs, th_max=1, th_min=0.3, max_iters=10, min_coef=2, max_coef=5,  margin=10, pt=1, pc=2, max_tries=3, shuffle=True):
    """Resuelve resolviendo mediante norma 2 y luego iterando hacia norma inf"""
    #indices de igualdades
    eqs = [n for n in range(b.shape[0]) if n not in idx_ineqs.astype(int)]
    #resultados actuales
    b_ = A @ q0
    # Se les asigna a las inecuaciones incumplidas el valor maximo permitido
    idx_ineqs_ok = np.intersect1d(np.where((b - b_) > 0), idx_ineqs).astype(int)
    b[idx_ineqs_ok] = b_[idx_ineqs_ok]
    
    #least squares
    sol, *_ = lsqr(A, b)
    #de var a subesp
    o = q0 - sol
    W = diags(q0).tocsr()

    thresh = np.linspace(th_min, th_max, max_iters)**pt
    coefs = np.linspace(min_coef**(1/2), max_coef**(1/2), max_iters)**pc
    res = check_sol(A, q0, b, show=False)
    t = 0
    i = 0
    while True:
        i += 1
        if i == max_tries+1:
            break
        print(f"Thresh {thresh[-i]:.2f} Coef {coefs[-i]:.2f} { {k:round(v, 2) for k,v in res.items()} } ")
        ort = orth((A @ W).T.todense())
        base = W @ ort
        x = o - base @ (base.T @ (W.power(-2) @ o))
        q_ = x + sol
        res_ = check_sol(A, q_, b, i, t, show=False, th1=th_max, th2=th_min)
        if not ((res_['Max'] < res['Max']) or (res_['Min'] > res['Min']) or (res_['Norm Aq-b'] < res['Norm Aq-b'] - margin)):
            if (t == 1) and shuffle:
                np.random.shuffle(thresh)
                np.random.shuffle(coefs)
            elif t == max_tries:
                print('Max tries reached.')
                break
            else:
                t += 1
                continue
        q = q_
        res = res_    
        idx = np.argwhere(np.abs(W.power(-1) @ (q_ - q0)) > thresh[-i]).flatten().astype(int)
        if idx.shape[0] == 0:
            continue
        W[idx, idx] = W[idx, idx] * (1 / coefs[-i]) 
    return q, res

In [20]:
def check_sol(A, q, b, Q = None, i = 0, t = 0, show=True, th1=1, th2=0.3, xlim=(-1, 1), ylim=None):
    if Q is None:
        Q = pd.read_csv(TMP_PATH + 'Q1.csv', sep=';')
    if A is None:
        A,q,b,idx_ineqs = prepare_vars()
    Q[neto_col + '_1'] = q
    Q['Dif'] = (1 - (Q[neto_col+'_1'] / Q[neto_col+'_w']))
    data = (np.linalg.norm(Q['Dif'], ord=2), Q['Dif'].max(), Q['Dif'].min(), Q['Dif'].mean(),   
                Q.loc[Q['Dif'].abs() > th1].shape[0], Q.loc[Q['Dif'].abs() > th2].shape[0], 
            Q[Q[neto_col+'_1'] < 0].shape[0], 
            np.linalg.norm(A @ q - b))
    names = f'Norma, Max, Min, Media, Mayores a {th1:.2f}, Mayores a {th2:.2f}, Negativos, Norm Aq-b, Iter {i}, Tries {t}'.split(', ')
    data = {k:v for k,v in zip(names, data)}
    if show:
        sns.distplot(Q['Dif'], kde=False, norm_hist=True)
        lim = max(np.abs([data['Max'], data['Min']])) + 0.1
        plt.xlim(-lim, lim)  
        if ylim: plt.ylim((0, ylim))
        print(*[str(t[0]) + ' ' + re.search(r"[\d\-]+\.*0*[.1-9]{0,2}", str(t[1])).group(0) for t in data.items()], sep = '\n')

    return data

In [114]:
def solve():
    t0 = time()
    print('Preparando optimización.')
    Q = pd.read_csv(TMP_PATH+'Q1.csv', sep=';')
    A, q0, b, idx_ineqs = prepare_vars()
    sol = solve_nras(A,q0.copy(),b,idx_ineqs)
    sol, data = solve_ultra(A, sol, b, idx_ineqs, max_tries=1)
    Q = pd.read_csv(TMP_PATH+'Q1.csv', sep=';')
    Q[neto_col + '_1'] = sol
    Q.to_csv(TMP_PATH + 'Q1.csv', sep=';', index=False)
    validar()
    ctypes.windll.user32.MessageBoxW(0, f"Distribución calculada. {time() - t0:.0f} segundos.",  "Terminado", 1)

In [ ]:
def ajustar():
    Q1 = pd.read_csv(TMP_PATH+'Q1.csv', sep=';')
    m = get_table(wb, 'Maestro', [c_col, 'Grupo Economico', cl_col, s_col, z_col, nom_cli_col, fle_col, cir_col], cast_num=False)

    coef_col = 'Coeficiente'
    wb = xw.Book(MASTER_PATH)
    ajust = get_table(wb, ajuste_sh, ajuste_cols)

    coefs = ajust[coef_col]
    ajust = ajust.drop(coef_col, axis=1)

    cols = m.columns | Q1.columns
    out = Q1.merge(m, on=[cl_col, s_col], how='left', suffixes=('_', ''))[cols]
    
    for i, row in ajust.iterrows():
        row = row.dropna()
        mask = np.logical_and.reduce([(out[c] == v) for c, v in row.items()])
        out.loc[mask, neto_col] = out[mask][neto_col] * coefs[i]
    
    A, x, b, idx_ineqs = [TMP_PATH + p for p in ['A.npz', 'x.gz', 'b.gz', 'idx_ineqs.gz']]
    A = sparse.load_npz(A).astype(bool)
    x = np.loadtxt(x).astype('float64')
    b = np.loadtxt(b).astype('int32')
    q, _ = solve(A, out[neto_col+'_1'].values, b, margin = 1000, lr = 0.01)
    
    write_sol(q)    
    check_constrains()

In [159]:
def solve_each(A, q, b, idx_ineqs, lr = 0.01, epochs = 10, lim=10, verbose=False, cvx = False, polish = False, precision=0.01, live = False):
    i = 0
    rows = A.shape[0]
    idx_eqs = [i for i in range(rows) if i not in idx_ineqs]
    for j in range(epochs):
        print('Epoch: ', j)
        for i in range(rows):
            a = A[i].toarray().ravel()
            lr_ = lr
            s = np.sign(b[i] - (q @ a))
            if np.all(abs(b[idx_eqs] - (A[idx_eqs] @ x)) < abs(b[idx_eqs])*precision ) & np.all(A[idx_ineqs] @ x <= (b[idx_ineqs] * (1 + precision))):
                    print('Todas las restricciones satisfechas.')
                    return q, 1
            while True:
                dif = b[i] - (q @ a)
                if abs(dif) < precision*b[i]:
                    if verbose: print('Optimizada','dif:', i, 'lr, ', lr_, s_, 'dif, ', dif, 'obj,', b[i])
                    break
                s_ = np.sign(dif)
                if s_ != s:
                    lr_ *= (1 - lr*2)
                q[a] *= (1 + (s_ * lr_))
                if verbose: print(',', end='.')
                s = s_
        if live:
            write_sol(q)
            validar()
    return q, 0

In [ ]:
%%time
import time

nras, proj = [], []
b = b.astype(float)
t0 = time.time()
for i in range(100):
    d = np.random.uniform(0.8, 1, size=b.shape[0])
    b_= b * d
    x1 = solve_nras(A,x,b_,idx_ineqs, verbose=False)
    nras.append(np.linalg.norm(b - A @ x1))
    t1 = time.time()
    print('nras,', t0 - t1)
    x2 = solve_exact(A,x,b_,idx_ineqs)
    proj.append(np.linalg.norm(b - A @ x2))
    t2 = time.time()
    print('proj,', t1 - t2) 